# 为什么要写单元测试

如果你在网上搜，可能会看到：

帮助理解需求、提高实现质量、方便代码重构，测试驱动开发......

![image](https://user-images.githubusercontent.com/57566630/204204454-3f35d6d8-a562-47bc-8ff0-5432c9b1fdf4.png)

事实上，每个学过编程语言的人，刷过 leetcode 之类算法题的人，都写过、跑过单元测试....

In [1]:
a = [1, 2, 3]
a.append(4)
print(f'The result of append is {a}')
a.extend([5, 6, 7])
print(f'The result of extend is {a}')

[1, 2, 3, 4]
[1, 2, 3, 4, 5, 6, 7]




# OpenMMLab 中单元测试规范

1. 单元测试文件夹均在根目录的 tests 文件夹内，子文件夹名以 `test_` 开头，例如 `test_runner`。

2. 单元测试文件名以 `test_` 开头，例如 `test_runner.py`。

3. 如果写类的单元测试，假设测试类名应该为 `Test{original_class_name}`，例如 Config 的测试类名为 `TestConfig`, Runner 的测试类名为 `TestRunner`。

4. 函数/方法的单元测试名应该为 `test_{original_fun_name}`，例如 `autocast` 的测试函数为 `test_autocast`

5. 单元测试应该既包括内部接口，也包括外部接口测试。像 Hook 之类与 Runner 强耦合的模块还需要配合 Runner 进行测试，例如 EMAHook 中的 [test_with_runner](./mmengine/tests/test_hooks/test_ema_hook.py)。目前 MMEngine 正在进行 Hook 部分的单元测试重构。

6. 每次提交代码应该保证单元测试率不会变低，除了抽象方法，理论上单元测试覆盖率 100%(条件触发的单元测试除外)。



## 单元测试测什么

### 输入测试

- 参数类型测试
    输入是否为期望的类型，例如是否为整型或者列表中的元素是否为整型

    ```python
    import pytest

    def add(a, b):
        if not (isinstance(a, (int, float)) and isinstance(b, (int, float))):
            raise ValueError('input should be numeric')
        return a + b
        
    def test_add():
        with pytest.raises(ValueError, 'input should be numeric'):
            add(1, '2')
        add(1, 2)
    ```

    例子 [test_registry](./mmengine/tests/test_registry/test_registry.py)

- 非法参数测试

    测试输入是否合法，例如要求除数不能为 0

    ```python
    import pytest

    def divide(a, b):
        if b == 0:
            raise ZeroDivisionError('divisor b cannot be 0')
        return a / b
        
    def test_divide():
        with pytest.raises(ZeroDivisionError, 'divisor b cannot be 0'):
            divide(1, 0)
        divide(1, 2)
    ```

- 边界测试

    测试输入是否符合上下界

    ```python
    import pytest

    def add_age(a, b):
        if a <=0 or b <= 0:
            raise Value('age should be greater than 0')
            
    def test_add_age():
        with pytest.raises(ZeroDivisionError, 'divisor b cannot be 0'):
            add_age(0, 1)
        
        with pytest.raises(ZeroDivisionError, 'divisor b cannot be 0'):
            add_age(1, 0)
        
        with pytest.raises(ZeroDivisionError, 'divisor b cannot be 0'):
            add_age(0, 0)
        
        assert add_age(1, 1) == 2
    ```


### 输出测试

测试输出是否符合预期

```python
import pytest

def add(a, b):
    if not (isinstance(a, (int, float)) and isinstance(b, (int, float))):
        raise ValueError('input should be numeric')
    return a + b
    
def test_add():
    assert add(1, 2) == 3
    assert add(-1, 2) == 1
    assert add(-1, 1) == 0
```

例子 [test_get_device](./mmengine/tests/test_device/test_device.py)

### 控制流测试

```python
@pytest.mark.skipif(torch is None, reason='requires torch library')
def test_assert_is_norm_layer():
    # case 1
    assert not testing.assert_is_norm_layer(nn.Conv3d(3, 64, 3))

    # case 2
    assert testing.assert_is_norm_layer(nn.BatchNorm3d(128))

    # case 3
    assert testing.assert_is_norm_layer(nn.GroupNorm(8, 64))

    # case 4
    assert not testing.assert_is_norm_layer(nn.Sigmoid())
```


## Unittest 基本概念

看过 MMEngine 单元测试的同学可能会知道，MMEngine 的单元测试处于 pytest 和 unittest 混用的状态：

- pytest: [Config](./mmengine/tests/test_config/test_config.py)
- unittest: [Runner](./mmengine/tests/test_runner/test_runner.py)

事实上，MMEngine 一开始是基于 pytest 写的单元测试，随着开发的不断进行，我们发现了 unittest 用起来更加的顺手也更加直观，后续的开发中也引入了 unittest。

> unittest 和 pytest 的区别： https://www.sitepoint.com/python-unit-testing-unittest-pytest/


### test fixture

每个单元测试之前要做啥，单元测试之后要做啥。例如每次单元测试之前我都需要创建一个临时目录，每次单元测试之后我都需要将其销毁。


例子 [test_fixture](./tests_demo/test_fixture.py)


### test case

单元测试的最小单位，一个 test case 通常包含一个或多个 test function。

例子 [test_case](./tests_demo/test_case.py)

### test suite & runner

Test fixture 和 Test case 用于描述 unittest 中单元测试的基本概念，而 suite 和 runner 则用于组织更大规模的单元测试。

例子 [test_suite](./tests_demo/test_suite.py)

看到这你或许会困惑，咋写个单元测试那么费劲呢，要了解 TestCase，TestSuite，TestLoader 以及 TestRunner 这么多概念，难受住，写单元测试已经很“委屈”了，还那么麻烦，不写了！。别急，我们还有 pytest！

![image](https://user-images.githubusercontent.com/57566630/204232783-f10dabc5-97ce-4b15-bc54-ab322de74f1a.png)

例子 [test_based_on_pytest](./tests_demo/test_based_on_pytest.py)

不仅定义单元测试简单，批量执行单元测试也更加的简单。




## 单元测试校验写法

- 基于 pytest 的测试函数

    1. 直接用 python 原生的 assert 进行校验

    2. 可以用 `with pytest.warns` 或者 `with pytest.raises` 去捕获异常

    3. 可以用 pytest 原生的 `tmp_path`, `capsys` 等一些 fixture。例如 [test_logger](./mmengine/tests/test_logging/test_logger.py)

- 基于 unittest 的测试类
    
    1. 使用 self.assertxxx 进行校验,例如 self.assertEqual, self.assertIs ... 等，具体可以参考 [unittest](https://docs.python.org/zh-cn/3/library/unittest.html#unittest.TestCase.assertEqual)

    2. 捕捉异常或者警告可以使用 `with self.assertRaisesRegex(...)` 或者 `with self.assertWarnsRegex(...)`，具体可以参考 [unittest](https://docs.python.org/zh-cn/3/library/unittest.html#unittest.TestCase.assertRaisesRegex)
    
    3. 捕捉日志输出可以使用 self.assertLogs(...)，可以参考 [test_optim_wrapper](./mmengine/tests/test_optim/test_optimizer/test_optimizer_wrapper.py)。

## 单元测试覆盖率

```shell
coverage run --branch --source mmengine -m pytest tests/test_dataset
coverage html
# coverage report
```


